<a href="https://colab.research.google.com/github/MadmanMarble/Planet_U-NET/blob/main/Planet_training_data_organization_032024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 38.1 MB/s eta 0:00:00


In [3]:
import rasterio
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import shutil
import hashlib

In [4]:
!mkdir /content/OneYear
!mkdir /content/OneYear/utm4
!mkdir /content/OneYear/utm4/images
!mkdir /content/OneYear/utm4/labels

!mkdir /content/OneYear/utm5
!mkdir /content/OneYear/utm5/images
!mkdir /content/OneYear/utm5/labels

!mkdir /content/TwoYears
!mkdir /content/TwoYears/utm4
!mkdir /content/TwoYears/utm4/images
!mkdir /content/TwoYears/utm4/labels

!mkdir /content/TwoYears/utm5
!mkdir /content/TwoYears/utm5/images
!mkdir /content/TwoYears/utm5/labels


# OneYear UTM 4 Data

In [5]:
!unzip /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4_updatedExports_march20/utm4_OneYear_stacked.zip -d /content/OneYear/utm4/images/

Archive:  /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4_updatedExports_march20/utm4_OneYear_stacked.zip
   creating: /content/OneYear/utm4/images/content/folder_stack/OneYear/
  inflating: /content/OneYear/utm4/images/content/folder_stack/OneYear/735_stacked.tif  
  inflating: /content/OneYear/utm4/images/content/folder_stack/OneYear/1009_stacked.tif  
  inflating: /content/OneYear/utm4/images/content/folder_stack/OneYear/1347_stacked.tif  
  inflating: /content/OneYear/utm4/images/content/folder_stack/OneYear/2342_stacked.tif  
  inflating: /content/OneYear/utm4/images/content/folder_stack/OneYear/1711_stacked.tif  
  inflating: /content/OneYear/utm4/images/content/folder_stack/OneYear/967_stacked.tif  
  inflating: /content/OneYear/utm4/images/content/folder_stack/OneYear/30_stacked.tif  
  inflating: /content/OneYear/utm4/images/content/folder_stack/OneYear/1276_stacked.tif  
  inflating: /content/OneYear/utm4/images/content/folder_stack/OneYear/103

In [7]:
# Define the source and destination directories
source_dir = "/content/OneYear/utm4"
destination_dir = "/content/OneYear/utm4/images"

# List all .tif files in the source directory
tif_files = [file for file in os.listdir(source_dir) if file.endswith(".tif")]

# Move each .tif file to the destination directory
for file in tif_files:
    # Define the source and destination file paths
    source_file = os.path.join(source_dir, file)
    destination_file = os.path.join(destination_dir, file)

    # Move the file to the destination directory
    shutil.move(source_file, destination_file)


In [9]:
# Define the folder containing .tif files
images_folder = '/content/OneYear/utm4/images'

# List all files in the folder
files = os.listdir(images_folder)

# Filter .tif files
tif_files = [file for file in files if file.endswith('.tif')]

# Count the number of .tif files
num_tif_files = len(tif_files)

# Print the number of .tif files
print(f'The number of .tif files in the folder is: {num_tif_files}')


The number of .tif files in the folder is: 2629


In [21]:
!unzip /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4_updatedExports_march20/utm4_2018_nwi_ccap_filled_07.zip -d /content/OneYear/utm4/labels/

Archive:  /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4_updatedExports_march20/utm4_2018_nwi_ccap_filled_07.zip
  inflating: /content/OneYear/utm4/labels/content/exports/nwi_ccap_filled/1818_nwi_ccap_filled.tif  
  inflating: /content/OneYear/utm4/labels/content/exports/nwi_ccap_filled/1250_nwi_ccap_filled.tif  
  inflating: /content/OneYear/utm4/labels/content/exports/nwi_ccap_filled/1832_nwi_ccap_filled.tif  
  inflating: /content/OneYear/utm4/labels/content/exports/nwi_ccap_filled/2389_nwi_ccap_filled.tif  
  inflating: /content/OneYear/utm4/labels/content/exports/nwi_ccap_filled/1660_nwi_ccap_filled.tif  
  inflating: /content/OneYear/utm4/labels/content/exports/nwi_ccap_filled/1471_nwi_ccap_filled.tif  
  inflating: /content/OneYear/utm4/labels/content/exports/nwi_ccap_filled/1201_nwi_ccap_filled.tif  
  inflating: /content/OneYear/utm4/labels/content/exports/nwi_ccap_filled/1378_nwi_ccap_filled.tif  
  inflating: /content/OneYear/utm4/labels/cont

In [23]:
# Define the source and destination directories
source_dir = "/content/OneYear/utm4/labels/content/exports/nwi_ccap_filled"
destination_dir = "/content/OneYear/utm4/labels"

# List all .tif files in the source directory
tif_files = [file for file in os.listdir(source_dir) if file.endswith(".tif")]

# Move each .tif file to the destination directory
for file in tif_files:
    # Define the source and destination file paths
    source_file = os.path.join(source_dir, file)
    destination_file = os.path.join(destination_dir, file)

    # Move the file to the destination directory
    shutil.move(source_file, destination_file)

In [24]:
# Define the folder containing .tif files
images_folder = '/content/OneYear/utm4/labels'

# List all files in the folder
files = os.listdir(images_folder)

# Filter .tif files
tif_files = [file for file in files if file.endswith('.tif')]

# Count the number of .tif files
num_tif_files = len(tif_files)

# Print the number of .tif files
print(f'The number of .tif files in the folder is: {num_tif_files}')


The number of .tif files in the folder is: 2629


# TwoYears UTM 4 Data

In [27]:
!unzip /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4_updatedExports_march20/utm4_TwoYears_stacked_03.zip -d /content/TwoYears/utm4/images/

Archive:  /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4_updatedExports_march20/utm4_TwoYears_stacked_03.zip
  inflating: /content/TwoYears/utm4/images/content/folder_stack/TwoYears/2342_stacked.tif  
  inflating: /content/TwoYears/utm4/images/content/folder_stack/TwoYears/2399_stacked.tif  
  inflating: /content/TwoYears/utm4/images/content/folder_stack/TwoYears/2210_stacked.tif  
  inflating: /content/TwoYears/utm4/images/content/folder_stack/TwoYears/2184_stacked.tif  
  inflating: /content/TwoYears/utm4/images/content/folder_stack/TwoYears/2124_stacked.tif  
  inflating: /content/TwoYears/utm4/images/content/folder_stack/TwoYears/2085_stacked.tif  
  inflating: /content/TwoYears/utm4/images/content/folder_stack/TwoYears/2554_stacked.tif  
  inflating: /content/TwoYears/utm4/images/content/folder_stack/TwoYears/2601_stacked.tif  
  inflating: /content/TwoYears/utm4/images/content/folder_stack/TwoYears/2622_stacked.tif  
  inflating: /content/TwoYears

In [28]:
# Define the source and destination directories
source_dir = "/content/TwoYears/utm4/images/content/folder_stack/TwoYears"
destination_dir = "/content/TwoYears/utm4/images"

# List all .tif files in the source directory
tif_files = [file for file in os.listdir(source_dir) if file.endswith(".tif")]

# Move each .tif file to the destination directory
for file in tif_files:
    # Define the source and destination file paths
    source_file = os.path.join(source_dir, file)
    destination_file = os.path.join(destination_dir, file)

    # Move the file to the destination directory
    shutil.move(source_file, destination_file)

In [29]:
# Define the folder containing .tif files
images_folder = '/content/TwoYears/utm4/images'

# List all files in the folder
files = os.listdir(images_folder)

# Filter .tif files
tif_files = [file for file in files if file.endswith('.tif')]

# Count the number of .tif files
num_tif_files = len(tif_files)

# Print the number of .tif files
print(f'The number of .tif files in the folder is: {num_tif_files}')

The number of .tif files in the folder is: 2605


# OneYear UTM 5 Data

In [30]:
!unzip /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5_updatedExports_march20/utm5_OneYear_stacked.zip -d /content/OneYear/utm5/images/

Archive:  /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5_updatedExports_march20/utm5_OneYear_stacked.zip
   creating: /content/OneYear/utm5/images/content/stack/OneYear/
  inflating: /content/OneYear/utm5/images/content/stack/OneYear/30_stacked.tif  
  inflating: /content/OneYear/utm5/images/content/stack/OneYear/61_stacked.tif  
  inflating: /content/OneYear/utm5/images/content/stack/OneYear/98_stacked.tif  
  inflating: /content/OneYear/utm5/images/content/stack/OneYear/167_stacked.tif  
  inflating: /content/OneYear/utm5/images/content/stack/OneYear/62_stacked.tif  
  inflating: /content/OneYear/utm5/images/content/stack/OneYear/65_stacked.tif  
  inflating: /content/OneYear/utm5/images/content/stack/OneYear/118_stacked.tif  
  inflating: /content/OneYear/utm5/images/content/stack/OneYear/59_stacked.tif  
  inflating: /content/OneYear/utm5/images/content/stack/OneYear/148_stacked.tif  
  inflating: /content/OneYear/utm5/images/content/stack/OneYear/2

In [31]:
# Define the source and destination directories
source_dir = "/content/OneYear/utm5/images/content/stack/OneYear"
destination_dir = "/content/OneYear/utm5/images"

# List all .tif files in the source directory
tif_files = [file for file in os.listdir(source_dir) if file.endswith(".tif")]

# Move each .tif file to the destination directory
for file in tif_files:
    # Define the source and destination file paths
    source_file = os.path.join(source_dir, file)
    destination_file = os.path.join(destination_dir, file)

    # Move the file to the destination directory
    shutil.move(source_file, destination_file)

In [32]:
# Define the folder containing .tif files
images_folder = '/content/OneYear/utm5/images'

# List all files in the folder
files = os.listdir(images_folder)

# Filter .tif files
tif_files = [file for file in files if file.endswith('.tif')]

# Count the number of .tif files
num_tif_files = len(tif_files)

# Print the number of .tif files
print(f'The number of .tif files in the folder is: {num_tif_files}')

The number of .tif files in the folder is: 229


In [33]:
!unzip /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5_updatedExports_march20/utm5_TwoYears_stacked.zip -d /content/TwoYears/utm5/images/

Archive:  /content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5_updatedExports_march20/utm5_TwoYears_stacked.zip
   creating: /content/TwoYears/utm5/images/content/stack/TwoYears/
  inflating: /content/TwoYears/utm5/images/content/stack/TwoYears/30_stacked.tif  
  inflating: /content/TwoYears/utm5/images/content/stack/TwoYears/61_stacked.tif  
  inflating: /content/TwoYears/utm5/images/content/stack/TwoYears/98_stacked.tif  
  inflating: /content/TwoYears/utm5/images/content/stack/TwoYears/167_stacked.tif  
  inflating: /content/TwoYears/utm5/images/content/stack/TwoYears/62_stacked.tif  
  inflating: /content/TwoYears/utm5/images/content/stack/TwoYears/65_stacked.tif  
  inflating: /content/TwoYears/utm5/images/content/stack/TwoYears/118_stacked.tif  
  inflating: /content/TwoYears/utm5/images/content/stack/TwoYears/59_stacked.tif  
  inflating: /content/TwoYears/utm5/images/content/stack/TwoYears/148_stacked.tif  
  inflating: /content/TwoYears/utm5/images/c

In [34]:
# Define the source and destination directories
source_dir = "/content/TwoYears/utm5/images/content/stack/TwoYears"
destination_dir = "/content/TwoYears/utm5/images"

# List all .tif files in the source directory
tif_files = [file for file in os.listdir(source_dir) if file.endswith(".tif")]

# Move each .tif file to the destination directory
for file in tif_files:
    # Define the source and destination file paths
    source_file = os.path.join(source_dir, file)
    destination_file = os.path.join(destination_dir, file)

    # Move the file to the destination directory
    shutil.move(source_file, destination_file)

In [35]:
# Define the folder containing .tif files
images_folder = '/content/TwoYears/utm5/images'

# List all files in the folder
files = os.listdir(images_folder)

# Filter .tif files
tif_files = [file for file in files if file.endswith('.tif')]

# Count the number of .tif files
num_tif_files = len(tif_files)

# Print the number of .tif files
print(f'The number of .tif files in the folder is: {num_tif_files}')

The number of .tif files in the folder is: 229


# For UTM 4 data, find the unmatched files

In [40]:
# Specify the folder path
folder_path = '/content/TwoYears/utm5/images'

# Function to extract the numeric part of the filename
def extract_numeric_part(filename):
    match = re.match(r"(\d+)", filename)
    return int(match.group()) if match else None

# List all files in the folder
files = os.listdir(folder_path)

# Extract numeric parts and sort
numeric_parts = [extract_numeric_part(file) for file in files if extract_numeric_part(file) is not None]
sorted_numeric_parts = sorted(numeric_parts)

# Find missing numbers
max_num = sorted_numeric_parts[-1]  # Assuming the largest number in the sequence is the last one
complete_range = set(range(1, max_num + 1))  # Start from 1 to max_num, inclusive
missing_numbers = complete_range - set(sorted_numeric_parts)

# Print missing numbers
print("Missing numbers:", sorted(list(missing_numbers)))

Missing numbers: []


In [44]:
# from collections import Counter

# # Are there any duplicate files?
# # No duplicate files found.

# # Specify the folder path
# folder_path = '/content/TwoYears/utm5/images'

# # Function to extract the numeric part of the filename
# def extract_numeric_part(filename):
#     match = re.match(r"(\d+)", filename)
#     return int(match.group()) if match else None

# # List all files in the folder
# files = os.listdir(folder_path)

# # Extract numeric parts
# numeric_parts = [extract_numeric_part(file) for file in files if extract_numeric_part(file) is not None]

# # Count occurrences of each numeric part
# numeric_parts_counts = Counter(numeric_parts)

# # Identify and print duplicates
# duplicates = {part: count for part, count in numeric_parts_counts.items() if count > 1}
# if duplicates:
#     print("Duplicate files (numeric part: count):")
#     for part, count in duplicates.items():
#         print(f"{part}: {count} times")
# else:
#     print("No duplicate files found.")

No duplicate files found.


In [45]:
!mkdir /content/OneYear/utm4/REMOVED

In [47]:
import os
import shutil

# Define the source and destination directories
source_dir = "/content/OneYear/utm4/images/"
destination_dir = "/content/OneYear/utm4/REMOVED/"

# List of specific file names to move
files_to_move = ["596_planet.tif", "597_planet.tif", "598_planet.tif", "599_planet.tif", "600_planet.tif", "601_planet.tif", "602_planet.tif", "603_planet.tif", "604_planet.tif", "605_planet.tif", "606_planet.tif", "607_planet.tif", "608_planet.tif", "609_planet.tif", "610_planet.tif", "611_planet.tif", "612_planet.tif", "613_planet.tif", "614_planet.tif", "1832_planet.tif", "1833_planet.tif", "1834_planet.tif"]

print(files_to_move)


# Iterate through the list of files to move
for file_name in files_to_move:
    # Construct the full path of the source file
    source_file = os.path.join(source_dir, file_name)

    # Check if the source file exists
    if os.path.exists(source_file):
        # Move the file to the destination directory
        shutil.move(source_file, destination_dir)
        print(f"File '{file_name}' moved successfully.")
    else:
        print(f"File '{file_name}' does not exist in the source directory.")


['596_planet.tif', '597_planet.tif', '598_planet.tif', '599_planet.tif', '600_planet.tif', '601_planet.tif', '602_planet.tif', '603_planet.tif', '604_planet.tif', '605_planet.tif', '606_planet.tif', '607_planet.tif', '608_planet.tif', '609_planet.tif', '610_planet.tif', '611_planet.tif', '612_planet.tif', '613_planet.tif', '614_planet.tif', '1832_planet.tif', '1833_planet.tif', '1834_planet.tif']
File '596_planet.tif' does not exist in the source directory.
File '597_planet.tif' does not exist in the source directory.
File '598_planet.tif' does not exist in the source directory.
File '599_planet.tif' does not exist in the source directory.
File '600_planet.tif' does not exist in the source directory.
File '601_planet.tif' does not exist in the source directory.
File '602_planet.tif' does not exist in the source directory.
File '603_planet.tif' does not exist in the source directory.
File '604_planet.tif' does not exist in the source directory.
File '605_planet.tif' does not exist in th